## DIA 126: Validar que la Nueva Categoría No Repita una Existente


Mejorar el formulario de carga de reportes para evitar que un usuario involuntariamente cree una categoría que ya existe (por error de tipeo, por ejemplo: "Mensual" vs "mensual").

🧩 1. Normalizar categorías existentes en Flask
En la vista cargar_reporte, convierte todas las categorías existentes a minúsculas para comparar correctamente:

python
Copiar
Editar
@app.route('/cargar', methods=['GET', 'POST'])
@login_required
def cargar_reporte():
    categorias = db.session.query(Reporte.categoria).filter_by(user_id=current_user.id).distinct().all()
    categorias = [c[0] for c in categorias if c[0]]
    categorias_lower = [c.lower() for c in categorias]

    if request.method == 'POST':
        titulo = request.form['titulo'].strip()
        nueva_categoria = request.form.get('nueva_categoria', '').strip()
        categoria_existente = request.form.get('categoria_existente', '').strip()

        # Determinar la categoría final
        if nueva_categoria:
            if nueva_categoria.lower() in categorias_lower:
                # Ya existe una similar: usar la categoría original que coincide
                categoria_final = next(c for c in categorias if c.lower() == nueva_categoria.lower())
            else:
                categoria_final = nueva_categoria
        else:
            categoria_final = categoria_existente

        nuevo_reporte = Reporte(
            titulo=titulo,
            categoria=categoria_final,
            user_id=current_user.id
        )
        db.session.add(nuevo_reporte)
        db.session.commit()

        flash("Reporte cargado con categoría válida.", "success")
        return redirect(url_for('historial_reportes'))

    return render_template("cargar_reporte.html", categorias=categorias)
🧩 2. Mejora opcional: mostrar una advertencia si la nueva categoría ya existe
Esto se puede hacer con JavaScript (opcional), pero por simplicidad lo resolvemos 100% en backend.